# 01. Data Preparation

This notebook loads the Penn World Table (PWT) 10.0 data, filters for the relevant OECD countries and time period, and calculates the necessary variables for the Solow growth model estimation.

**Input**: `../data/raw/pwt100.xlsx`
**Output**: `../data/processed/mrw_clean_data.csv`

In [1]:
import pandas as pd
import numpy as np
import os

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## 1. Load Data

In [2]:
# Define file paths
raw_data_path = '../data/raw/pwt100.xlsx'
processed_data_dir = '../data/processed'

# Ensure processed directory exists
os.makedirs(processed_data_dir, exist_ok=True)

# Load the dataset
try:
    # Assuming the data is in the 'Data' sheet
    df_raw = pd.read_excel(raw_data_path, sheet_name='Data')
    print(f"Data loaded successfully: {df_raw.shape}")
except Exception as e:
    print(f"Error loading data: {e}")

Data loaded successfully: (12810, 52)


## 2. Filter Countries and Years

We select 28 OECD countries as specified in the project scope and cover the period 1960-2019.

In [3]:
# OECD Countries List
oecd_countries = [
    'AUT', 'BEL', 'DNK', 'FRA', 'DEU', 'GRC', 'ISL', 'IRL', 'ITA', 'LUX',
    'NLD', 'NOR', 'POL', 'PRT', 'ESP', 'SWE', 'CHE', 'TUR', 'CZE', 'HUN',
    'FIN', 'KOR', 'JPN', 'AUS', 'NZL', 'CAN', 'USA', 'MEX'
]

# Exclude specific countries if necessary (based on original code logic)
exclude_countries = ['SVK', 'SVN', 'CHL', 'COL'] 
final_countries = [c for c in oecd_countries if c not in exclude_countries]

# Time Period
start_year = 1960
end_year = 2019

# Filter Data
df = df_raw.copy()
df = df[df['countrycode'].isin(final_countries)]
df = df[(df['year'] >= start_year) & (df['year'] <= end_year)]

print(f"Filtered data shape: {df.shape}")
print(f"Countries included: {df['countrycode'].unique()}")
print(f"Years covered: {df['year'].min()} - {df['year'].max()}")

Filtered data shape: (1680, 52)
Countries included: ['AUS' 'AUT' 'BEL' 'CAN' 'CHE' 'CZE' 'DEU' 'DNK' 'ESP' 'FIN' 'FRA' 'GRC'
 'HUN' 'IRL' 'ISL' 'ITA' 'JPN' 'KOR' 'LUX' 'MEX' 'NLD' 'NOR' 'NZL' 'POL'
 'PRT' 'SWE' 'TUR' 'USA']
Years covered: 1960 - 2019


## 3. Variable Calculation

We calculate the required variables for the Solow model:
- **n**: Population growth rate
- **g + delta**: Assumed to be 0.05 (5%)
- **s_k**: Investment rate (physical capital share)
- **s_h**: Human capital
- **y**: GDP per working-age person (or per capita, depending on labor definition)

In [4]:
# get pop from WDI Data
df_wdi = pd.read_excel('../data/raw/WDIData.xlsx', sheet_name='Data')

import re
df_wdi.columns = list(df_wdi.columns[:4]) + [re.sub(r' \[.*\]', '', col) for col in df_wdi.columns[4:]]

year_cols = [int(col) for col in df_wdi.columns[4:] if col.isdigit() and start_year <= int(col) <= end_year]

id_vars = df_wdi.columns[:4].tolist()
df_pop = df_wdi.melt(id_vars=id_vars, var_name='Year', value_name='pop')

df_pop = df_pop.rename(columns={'Country Code': 'countrycode', 'Year': 'year', 'pop': 'pop_wdi'})

df_pop['year'] = df_pop['year'].astype(int)

df_pop = df_pop[df_pop['year'].astype(int).isin(year_cols)]

print(df_pop)

                                            Series Name     Series Code    Country Name countrycode  year  pop_wdi
0                          Population ages 15-64, total  SP.POP.1564.TO     Afghanistan         AFG  1960  4990044
1                          Population ages 15-64, total  SP.POP.1564.TO         Albania         ALB  1960   858107
2                          Population ages 15-64, total  SP.POP.1564.TO         Algeria         DZA  1960  5865620
3                          Population ages 15-64, total  SP.POP.1564.TO  American Samoa         ASM  1960     9703
4                          Population ages 15-64, total  SP.POP.1564.TO         Andorra         AND  1960     6275
...                                                 ...             ...             ...         ...   ...      ...
16255                                               NaN             NaN             NaN         NaN  2019      NaN
16256                                               NaN             NaN         

In [5]:
df = pd.merge(df, df_pop[['countrycode', 'year', 'pop_wdi']], on=['countrycode', 'year'], how='left')

In [ ]:
# Aggregating data for cross-sectional regression
# We need values at the start and end, and averages for the period.

def calculate_growth_vars(group):
    group = group.sort_values('year')
    
    start_data = group.iloc[0]
    end_data = group.iloc[-1]
    
    period_len = end_data['year'] - start_data['year']
    
    labor_measure = 'pop_wdi'
    
    y_start = start_data['rgdpna'] / start_data[labor_measure]
    y_end = end_data['rgdpna'] / end_data[labor_measure]
    
    # 2. Population growth (n)
    n = (end_data['pop_wdi'] / start_data['pop_wdi']) ** (1/period_len) - 1
    
    # 3. Investment rate (s_k)
    # csh_i: Share of gross capital formation at current PPPs
    # We take the average over the period
    s_k = group['csh_i'].mean()
    
    # 4. Human Capital (s_h or hc)
    # PWT 'hc' index: Index of human capital per person, based on years of schooling and returns to education
    hc_avg = group['hc'].mean()
    
    return pd.Series({
        'y_start': y_start,
        'y_end': y_end,
        'n': n,
        's_k': s_k,
        'hc': hc_avg
    })

df_growth = df.groupby('countrycode').apply(calculate_growth_vars)

# Log transformations
df_growth['ln_y_start'] = np.log(df_growth['y_start'])
df_growth['ln_y_end'] = np.log(df_growth['y_end'])
df_growth['growth_rate'] = df_growth['ln_y_end'] - df_growth['ln_y_start']

# Constants
g_delta = 0.05
df_growth['ln_n_g_delta'] = np.log(df_growth['n'] + g_delta)
df_growth['ln_s_k'] = np.log(df_growth['s_k'])
df_growth['ln_hc'] = np.log(df_growth['hc'])

print("Processed Variables Sample:")
print(df_growth.head())

Processed Variables Sample:
              y_start     y_end         n       s_k        hc  ln_y_start  ln_y_end  growth_rate  ln_n_g_delta    ln_s_k     ln_hc
countrycode                                                                                                                       
AUS          0.029428  0.079417  0.016534  0.285702  3.278253   -3.525795 -2.533042     0.992753     -2.710044 -1.252808  1.187311
AUT          0.021197  0.080564  0.004156  0.271485  2.937786   -3.853892 -2.518699     1.335193     -2.915879 -1.303850  1.077656
BEL          0.019615  0.072548  0.003794  0.295502  2.741410   -3.931450 -2.623509     1.307940     -2.922600 -1.219079  1.008472
CAN          0.029564  0.074936  0.014819  0.248156  3.247813   -3.521184 -2.591122     0.930062     -2.736160 -1.393696  1.177982
CHE          0.051479  0.113872  0.008266  0.352727  3.407427   -2.966588 -2.172677     0.793911     -2.842731 -1.042061  1.225957


C:\Users\User\AppData\Local\Temp\ipykernel_14488\3542519431.py:39: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_growth = df.groupby('countrycode').apply(calculate_growth_vars)


## 4. Save Processed Data

In [8]:
output_path = os.path.join(processed_data_dir, 'mrw_clean_data.csv')
df_growth.to_csv(output_path)
print(f"Saved processed data to {output_path}")

Saved processed data to ../data/processed\mrw_clean_data.csv
